# Desafio 5

## Pre-desarrollo

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
import missingno as msngo
import re

In [3]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["figure.figsize"] = (10, 6)

In [11]:
BASE_PATH = './Datos/'
EARNINGS_FINELAME = 'earnings.csv'
df = pd.read_csv(BASE_PATH+EARNINGS_FINELAME, )
df.head()

,height,weight,male,earn,earnk,ethnicity,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
0,74,210.0,1,50000.0,50.0,White,16.0,16.0,16.0,3,3,2.0,0.0,0.0,45
1,66,125.0,0,60000.0,60.0,White,16.0,16.0,16.0,6,5,1.0,0.0,0.0,58
2,64,126.0,0,30000.0,30.0,White,16.0,16.0,16.0,8,1,2.0,1.0,1.0,29
3,65,200.0,0,25000.0,25.0,White,17.0,17.0,NaN,8,1,2.0,0.0,0.0,57
4,63,110.0,0,50000.0,50.0,Other,16.0,16.0,16.0,5,6,2.0,0.0,0.0,91


## Desarrollo

### Problema 1
Considerando las variables earn, height y age, vamos a suponer que nuestro dataset fuera una población completa. (Prepara los datos adecuadamente).

#### A
. Realiza 5 pruebas con muestras de tamaño n = 25, en cada caso, con nivel s
de significancia diferentes pero inferiores a 0,1, para confirmar o rechazar  as
siguientes hipótesis considerando las alternativas. Crea para ello una fór ula
que reciba los parámetros adecuados y responda "con nivel de confianz  del
...%, se acepta la hipótesis nula/alternaiva".

In [41]:
def make_hipotesis_test_of_media(data, alpha, mu, form, n=25):
    confidence = (1-alpha)*100
    sample_data = np.random.choice(data, n)
    t_stat, p_value = stats.ttest_1samp(sample_data, mu, alternative=form)

    results = {
        'two-sided': f"With a confidence level of {confidence}% we can accept the alternative hypothesis (mean is different than {mu})",
        'greater': f"With a confidence level of {confidence}% we can accept the alternative hypothesis (mean is bigger than {mu})",
        'less': f"With a confidence level of {confidence}% we can accept the alternative hypothesis (mean is lower than {mu})",
        'reject': f"With a confidence level of {confidence}% we can accept the null hypothesis (mean is {mu})"
    }
    # Revisar si el signo esta bien
    if p_value > alpha:
        form = 'reject'
    print(results.get(form))
    print(f"t_stat: {t_stat}")
    print(f"p_value: {p_value}\n\n")
    

In [47]:
cases = {
    'earn':[(20000, 'two-sided'), (35000,'two-sided'), (25000, 'greater'), (32000, 'less')],
    'height':[(55, 'two-sided'), (68,'two-sided'), (63, 'greater'), (70, 'less')],
    'age':[(44, 'two-sided'), (56,'two-sided'), (51, 'greater'), (62, 'less')]
}

for key, values in cases.items(): 
    print("Evaluating cases for variable :", key)
    for value in values:
        alpha = round(np.random.uniform(0.01, 0.1), 2)
        make_hipotesis_test_of_media(df[key].dropna(), alpha, value[0], form=value[1])

Evaluating cases for variable : earn
With a confidence level of 98.0% we can accept the null hypothesis (mean is 20000)
t_stat: 0.5109552329707664
p_value: 0.6140508304415906


With a confidence level of 91.0% we can accept the alternative hypothesis (mean is different than 35000)
t_stat: -1.9750233189766377
p_value: 0.05987620889015563


With a confidence level of 96.0% we can accept the null hypothesis (mean is 25000)
t_stat: -2.773811308766445
p_value: 0.9947245809083174


With a confidence level of 97.0% we can accept the alternative hypothesis (mean is lower than 32000)
t_stat: -4.554961915006445
p_value: 6.439000307576084e-05


Evaluating cases for variable : height
With a confidence level of 95.0% we can accept the alternative hypothesis (mean is different than 55)
t_stat: 15.047707141471978
p_value: 1.0131168162300782e-13


With a confidence level of 93.0% we can accept the null hypothesis (mean is 68)
t_stat: -0.696785684263256
p_value: 0.49263132364792306


With a confidence 

#### B
Calcula la media poblacional para cada variable. ¿Son correctos losresultados obtenidos por tus pruebas?

In [45]:
print(f"The mean of earn column is {round(df['earn'].mean())}") 

The mean of earn column is 21147


We have the mean of the column is 21147, that why in the first (200000) and third (250000) case accept the null hypothesis, but we said in the fourth case that its lower than 32000

In [48]:
print(f"The mean of height column is {round(df['height'].mean())}") 

The mean of height column is 67


We have the mean of the height column is 67, in this case accept the second case, similar to 68, we accept that its bigger than 63 and lower tham 70

In [52]:
print(f"The mean of age column is {round(df['age'].mean())}") 

The mean of age column is 43


We have the mean of the age column is 43, in this case accept the first case, similar to 44, we accept that lower than 62, the case of accept that its 51 probably its because the confidence level

### Problema 2
Crea una función que estime la proporción de hombres en el dataset, considerando
una muestra de 50 individuos, con niveles de significancia de 0,05 y 0,01. Escribe con
palabras tus resultados ejecutar la función e interpreta. Compara con la proporción
real.


In [75]:
def estimate_proporcion_in_sample(data, filter_value, alpha_min, alpha_max, n_sample):
    real_proportion = len(data[data == filter_value])/len(data)
    sample_data = np.random.choice(data, n_sample)
    alpha = np.round(np.random.uniform(alpha_min, alpha_max), 2)
    confidence = (1-alpha)*100
    quantity_filter_sample = len(sample_data[sample_data == filter_value])
    confidence_interval = sm.stats.proportion_confint(quantity_filter_sample, n_sample, alpha, method='wilson')
    print(f"Quantity of male in the sample data {quantity_filter_sample}")
    print(f"The confidence interval is between {confidence_interval[0]} and {confidence_interval[1]} with a confidence of {confidence}%")
    print(f"The real proportion of male its {real_proportion}")

estimate_proporcion_in_sample(df['male'], 1, 0.01, 0.05, 50)

Quantity of male in the sample data 16
The confidence interval is between 0.19775980350081804 and 0.4732284461959264 with a confidence of 97.0%
The real proportion of male its 0.3716960352422907


We can consider that its correct, the real proportion its inside of the confidence interval

### Problema 3
Considerando el dataset como una muestra, verifica si el género (male=1 significa ‘hombre”) influye sobre el sueldo “earn” de las personas. Explica y justifica tu procedimiento.

In [76]:
sueldos_h = df[df['male'] == 1]["earn"].dropna()
sueldos_m = df[df['male'] == 0]["earn"].dropna()

In [78]:
# realizamos un test de hipotesis para variables independientes
t_stat, p_value = stats.ttest_ind(sueldos_h, sueldos_m, equal_var=False)

alpha = 0.03

if p_value < alpha:
    print("Se rechaza hipotesis nula")
else:
    print("No se rechaza la hipotesis nula")
print(f"T Stat : {t_stat}")
print(f"P_value : {p_value}")

Se rechaza hipotesis nula
T Stat : 11.86459381795286
P_value : 2.7373648004575493e-30


En otras palabras, se rechaza la influencia en el genero en el sueldo